In [5]:
import pandas as pd

# Cargar archivos CSV desde el mismo directorio donde está tu notebook
dim_category = pd.read_csv('DIM_CATEGORY.csv')
fact_sales = pd.read_csv('FACT_SALES.csv')

# Cargar archivos Excel
dim_product = pd.read_excel('DIM_PRODUCT.xlsx')
dim_segment = pd.read_excel('DIM_SEGMENT.xlsx')
dim_calendar = pd.read_excel('DIM_CALENDAR.xlsx')


In [31]:
import os

# Ver todo el directorio actual
print(os.listdir())


['.anaconda', '.conda', '.condarc', '.continuum', '.ipynb_checkpoints', '.ipython', '.jupyter', '.matplotlib', '.ms-ad', '.virtual_documents', 'anaconda3', 'AppData', 'Configuración local', 'Contacts', 'Cookies', 'Datos de programa', 'datos_consolidados.xlsx', 'Desktop', 'DIM_CALENDAR.xlsx', 'DIM_CATEGORY.csv', 'DIM_PRODUCT.xlsx', 'DIM_SEGMENT.xlsx', 'Documentos - Acceso directo.lnk', 'Documents', 'Downloads', 'Entorno de red', 'FACT_SALES.csv', 'Favorites', 'Impresoras', 'IntelGraphicsProfiles', 'Links', 'Menú Inicio', 'Microsoft', 'Mis documentos', 'Music', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{73acdb93-e298-11ef-ae0c-b7704cacf9ce}.TM.blf', 'NTUSER.DAT{73acdb93-e298-11ef-ae0c-b7704cacf9ce}.TMContainer00000000000000000001.regtrans-ms', 'NTUSER.DAT{73acdb93-e298-11ef-ae0c-b7704cacf9ce}.TMContainer00000000000000000002.regtrans-ms', 'ntuser.ini', 'OneDrive', 'Pictures', 'Plantillas', 'Reciente', 'Saved Games', 'Searches', 'SendTo', 'SYB66_246_202310_Population G

In [7]:
# Revisar DIM_CATEGORY
print("=== DIM_CATEGORY ===")
print(dim_category.head())
print(dim_category.info())
print(dim_category.describe(include='all'))

# Revisar FACT_SALES
print("\n=== FACT_SALES ===")
print(fact_sales.head())
print(fact_sales.info())
print(fact_sales.describe(include='all'))

# Revisar DIM_PRODUCT
print("\n=== DIM_PRODUCT ===")
print(dim_product.head())
print(dim_product.info())
print(dim_product.describe(include='all'))

# Revisar DIM_SEGMENT
print("\n=== DIM_SEGMENT ===")
print(dim_segment.head())
print(dim_segment.info())
print(dim_segment.describe(include='all'))

# Revisar DIM_CALENDAR
print("\n=== DIM_CALENDAR ===")
print(dim_calendar.head())
print(dim_calendar.info())
print(dim_calendar.describe(include='all'))


=== DIM_CATEGORY ===
   ID_CATEGORY                        CATEGORY
0            1  FABRIC TREATMENT and SANIT\r\n
1            2                       AIR CARE 
2            3                    LAVAVAJILLAS
3            4                MEGA SUPERFICIES
4            5             LAVATORY CARE & BRC
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID_CATEGORY  5 non-null      int64 
 1   CATEGORY     5 non-null      object
dtypes: int64(1), object(1)
memory usage: 212.0+ bytes
None
        ID_CATEGORY                        CATEGORY
count      5.000000                               5
unique          NaN                               5
top             NaN  FABRIC TREATMENT and SANIT\r\n
freq            NaN                               1
mean       3.000000                             NaN
std        1.581139                             NaN
min      

In [30]:
# LIMPIEZA BÁSICA DE TEXTOS
dim_category['CATEGORY'] = dim_category['CATEGORY'].astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)

fact_sales['ITEM_CODE'] = fact_sales['ITEM_CODE'].astype(str).str.strip()
dim_product['ITEM'] = dim_product['ITEM'].astype(str).str.strip()

text_cols = ['MANUFACTURER', 'BRAND', 'ITEM_DESCRIPTION', 'FORMAT', 'ATTR1', 'ATTR2', 'ATTR3']
for col in text_cols:
    if col in dim_product.columns:
        dim_product[col] = dim_product[col].astype(str).str.strip()

if 'SEGMENT' in dim_segment.columns:
    dim_segment['SEGMENT'] = dim_segment['SEGMENT'].astype(str).str.strip()


In [9]:
# Mostrar cuántos nulos hay por DataFrame
print("Nulos en DIM_PRODUCT:\n", dim_product.isnull().sum())
print("Nulos en DIM_SEGMENT:\n", dim_segment.isnull().sum())
print("Nulos en DIM_CALENDAR:\n", dim_calendar.isnull().sum())
print("Nulos en FACT_SALES:\n", fact_sales.isnull().sum())
print("Nulos en DIM_CATEGORY:\n", dim_category.isnull().sum())

# Eliminar filas con ITEM nulo en DIM_PRODUCT (clave primaria)
dim_product = dim_product.dropna(subset=['ITEM'])

# Rellenar nulos con "NO DEFINIDO" en otras columnas
dim_product.fillna('NO DEFINIDO', inplace=True)
dim_segment.fillna('NO DEFINIDO', inplace=True)
dim_calendar.fillna(method='ffill', inplace=True)  # Propaga hacia adelante si faltan fechas


Nulos en DIM_PRODUCT:
 MANUFACTURER        0
BRAND               0
ITEM                0
ITEM_DESCRIPTION    0
CATEGORY            0
FORMAT              0
ATTR1               0
ATTR2               0
ATTR3               0
dtype: int64
Nulos en DIM_SEGMENT:
 CATEGORY    0
ATTR1       0
ATTR2       0
ATTR3       1
FORMAT      0
SEGMENT     0
dtype: int64
Nulos en DIM_CALENDAR:
 WEEK           0
YEAR           0
MONTH          0
WEEK_NUMBER    0
DATE           0
dtype: int64
Nulos en FACT_SALES:
 WEEK                           0
ITEM_CODE                      0
TOTAL_UNIT_SALES               0
TOTAL_VALUE_SALES              0
TOTAL_UNIT_AVG_WEEKLY_SALES    0
REGION                         0
dtype: int64
Nulos en DIM_CATEGORY:
 ID_CATEGORY    0
CATEGORY       0
dtype: int64


C:\Users\Rafael\AppData\Local\Temp\ipykernel_10780\1827218572.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dim_calendar.fillna(method='ffill', inplace=True)  # Propaga hacia adelante si faltan fechas


In [10]:
# Verificar y eliminar duplicados
dim_category.drop_duplicates(inplace=True)
fact_sales.drop_duplicates(inplace=True)
dim_product.drop_duplicates(inplace=True)
dim_segment.drop_duplicates(inplace=True)
dim_calendar.drop_duplicates(inplace=True)


In [11]:
df_ventas_productos = fact_sales.merge(
    dim_product,
    left_on='ITEM_CODE',
    right_on='ITEM',
    how='left'
)


In [12]:
df_ventas_productos_categoria = df_ventas_productos.merge(
    dim_category,
    left_on='CATEGORY',
    right_on='ID_CATEGORY',
    how='left'
)


In [16]:
# Renombrar la columna de categoría para que coincida con DIM_SEGMENT
df_ventas_productos_categoria.rename(columns={'CATEGORY_x': 'CATEGORY'}, inplace=True)

# Ahora sí se puede hacer la unión
df_completo = df_ventas_productos_categoria.merge(
    dim_segment,
    on=['CATEGORY', 'ATTR1', 'ATTR2', 'ATTR3', 'FORMAT'],
    how='left'
)


In [17]:
df_completo = df_completo.merge(
    dim_calendar,
    on='WEEK',
    how='left'
)


In [29]:
if 'DATE' in df_completo.columns:
    df_completo['DATE'] = pd.to_datetime(df_completo['DATE'], errors='coerce')


In [20]:
cat_cols = ['REGION', 'MANUFACTURER', 'BRAND', 'CATEGORY', 'FORMAT', 'ATTR1', 'ATTR2', 'ATTR3', 'SEGMENT']
for col in cat_cols:
    if col in df_completo.columns:
        df_completo[col] = df_completo[col].astype('category')


In [21]:
df_completo['YEAR'] = df_completo['DATE'].dt.year
df_completo['MONTH'] = df_completo['DATE'].dt.month
df_completo['WEEK_NUM'] = df_completo['DATE'].dt.isocalendar().week


In [28]:
df_completo['AVG_UNIT_PRICE'] = df_completo['TOTAL_VALUE_SALES'] / df_completo['TOTAL_UNIT_SALES']


In [23]:
ventas_por_categoria = df_completo.groupby(['YEAR', 'CATEGORY'])['TOTAL_VALUE_SALES'].sum().reset_index()


C:\Users\Rafael\AppData\Local\Temp\ipykernel_10780\4155473083.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ventas_por_categoria = df_completo.groupby(['YEAR', 'CATEGORY'])['TOTAL_VALUE_SALES'].sum().reset_index()


In [24]:
print(df_completo[['DATE', 'YEAR', 'MONTH', 'WEEK_NUM', 'AVG_UNIT_PRICE']].head())


        DATE  YEAR  MONTH  WEEK_NUM  AVG_UNIT_PRICE
0 2022-08-28  2022      8        34       23.166667
1 2022-08-28  2022      8        34      239.258727
2 2022-08-28  2022      8        34       49.211359
3 2022-08-28  2022      8        34       67.318182
4 2022-08-28  2022      8        34       89.758959


In [27]:
# Guardar archivo Excel
df_completo.to_excel('datos_consolidados.xlsx', index=False)


In [26]:
import os
print(os.listdir())


['.anaconda', '.conda', '.condarc', '.continuum', '.ipynb_checkpoints', '.ipython', '.jupyter', '.matplotlib', '.ms-ad', '.virtual_documents', 'anaconda3', 'AppData', 'Configuración local', 'Contacts', 'Cookies', 'Datos de programa', 'datos_consolidados.xlsx', 'Desktop', 'DIM_CALENDAR.xlsx', 'DIM_CATEGORY.csv', 'DIM_PRODUCT.xlsx', 'DIM_SEGMENT.xlsx', 'Documentos - Acceso directo.lnk', 'Documents', 'Downloads', 'Entorno de red', 'FACT_SALES.csv', 'Favorites', 'Impresoras', 'IntelGraphicsProfiles', 'Links', 'Menú Inicio', 'Microsoft', 'Mis documentos', 'Music', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{73acdb93-e298-11ef-ae0c-b7704cacf9ce}.TM.blf', 'NTUSER.DAT{73acdb93-e298-11ef-ae0c-b7704cacf9ce}.TMContainer00000000000000000001.regtrans-ms', 'NTUSER.DAT{73acdb93-e298-11ef-ae0c-b7704cacf9ce}.TMContainer00000000000000000002.regtrans-ms', 'ntuser.ini', 'OneDrive', 'Pictures', 'Plantillas', 'Reciente', 'Saved Games', 'Searches', 'SendTo', 'SYB66_246_202310_Population G